### My study note for pre-processing the text_based data for Classifications

> If we have text_based data including names and descriptions of the products, and before we build and fit a classifier, we need to deal with the text_based data.<br>

> The following is the methods and codings for preprecessing such data-set

### 1.Filter data and Stem Word

> It has been decided that products with names or description other than in English will not be considered. The "langdetect" package does this work. Furthermore, it is reasonable to expect that the names and description not to be too short, like more than 5 words.

In [1]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def filterWrapper(textAndCate):
    try:
        if detect(textAndCate[0])=='en' and len(textAndCate[0])>5:
            return True
        else:
            return False
    except LangDetectException:
        return False

> It makes sense to stem the text. That means we will treat words in its different forms as one, for instance, “shoe” and “shoes”. We can utilize the "nltk (natural language toolkit)" package to accomplish this.

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

stemmer = SnowballStemmer('english')

def stemAndConCat(text):
    stemmedText = ''
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    for word in words:
        if word not in stop_words:
            stemmedText = stemmedText + stemmer.stem(word)+''
    return stemmedText

> we now have a list comprised of stemmed text. We also have the corresponding parent categories (their indexes), since we would like to begin with the classifier on the higher level, i.e. classifying products into parent categories. The list is shuffled before being split.<br>

> Next, we transform the text into vectors using the "TF-IDF algorithm", before feeding it into the model. This is accomplished by using the "pipeline", which handles a series of transformation. To speed up the training process later, we can cache the transformation. <br>

> All of the algorithms are implemented in the "sklearn" package, and we can simply import and call the APIs.

In [3]:
#shuffling
#Suppose My_Data is the name of my sampling data set, 
#in order to run code successfully, Here randomly define a data frame
import pandas as pd
from random import shuffle
My_Data = pd.DataFrame(data=[['A',"B","C","D","E","Text"] for i in range(1,6)])

data_target_list = []
for cate_tuple in My_Data.itertuples(): 
    data_target_list.append(((cate_tuple[1]+''+cate_tuple[2]).lower().replace('description',' '), cate_tuple[4]))#4 is parent categ.

data_target_list = list(filter(filterWrapper, data_target_list))
sizeOfList = len(data_target_list)
shuffle(data_target_list)
data_list = [stemAndConcat(item[0]) for item in data_target_list]
target_list = [item[1] for item in data_target_list]

In [4]:
#Pipelining Transformation
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tempfile import mkdtemp

cachedir = mkdtemp()
def getPipeline(clf):
    count_vect = CountVectorizer(ngram_range = (1,2), stop_words = 'english', analyzer = 'word')
    tfidfTransformer = TfidTransformer()
    return Pipeline([('count', count_vect), ('tf', tfidTransformer), ('clf', clf)], memory=cachedir)

> Now, we finish preprocessing the data. Then, we could use Classification methods, like K-means, Hierarchical classification algorithms to train a classifiers